In [68]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
df = pd.read_csv("./diabetes.csv")

In [5]:
df

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,negative
764,2,122,70,27,0,36.8,27,negative
765,5,121,72,23,112,26.2,30,negative
766,1,126,60,0,0,30.1,47,positive


In [13]:
df.describe()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,81.000000


In [23]:
 x = df.iloc[:, 0:-1].values
 y_string = list(df.iloc[:, -1])

In [29]:
y_int = []

for s in y_string:
    if s == "positive":
        y_int.append(1)
    else:
        y_int.append(0)

In [39]:
y = np.array(y_int, dtype="float64")

In [44]:
sc = StandardScaler()
x = sc.fit_transform(x)

In [58]:
x = torch.tensor(x)
y =  torch.tensor(y).unsqueeze(1)

C:\Users\naolz\AppData\Local\Temp\ipykernel_3888\3500401618.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
C:\Users\naolz\AppData\Local\Temp\ipykernel_3888\3500401618.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y =  torch.tensor(y).unsqueeze(1)


In [60]:
y.shape

torch.Size([768, 1])

In [88]:
class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return len(self.x)
        
        

In [90]:
dataset = Dataset(x, y)

In [92]:
len(dataset)

768

In [94]:
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

In [96]:
train_loader

In [98]:
for (x, y) in train_loader:
    print("For one iteration(batch), there is:")
    print("Data: {}".format(x.shape))
    print("Labels: {}".format(y.shape))
    break

For one iteration(batch), there is:
Data: torch.Size([32, 7])
Labels: torch.Size([32, 1])


In [108]:
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features,5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [110]:
net = Model(7, 1)
craterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(net.parameters(), lr=0.1,momentum=0.9)

C:\Users\naolz\anaconda3\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [120]:
epochs = 200
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        #forward prop
        outputs = net(inputs)
        #loss calculation
        loss = craterion(outputs, labels)
        #clear gradient buffer
        optimizer.zero_grad()
        #back prop
        loss.backward()
        #update weight
        optimizer.step()

    #Accuracy callculation
    output = (outputs > 0.5).float()
    accuracy = (output == labels).float().mean()
    print("Epoch {}/{}, Loss: {:3f}, Accuracy: {:3f}".format(epoch + 1, epochs, loss, accuracy))
        
        
        
        

Epoch 1/200, Loss: 0.461031, Accuracy: 0.781250
Epoch 2/200, Loss: 0.419410, Accuracy: 0.843750
Epoch 3/200, Loss: 0.511564, Accuracy: 0.718750
Epoch 4/200, Loss: 0.398053, Accuracy: 0.875000
Epoch 5/200, Loss: 0.426145, Accuracy: 0.781250
Epoch 6/200, Loss: 0.417162, Accuracy: 0.843750
Epoch 7/200, Loss: 0.353794, Accuracy: 0.812500
Epoch 8/200, Loss: 0.474728, Accuracy: 0.812500
Epoch 9/200, Loss: 0.462345, Accuracy: 0.750000
Epoch 10/200, Loss: 0.416253, Accuracy: 0.843750
Epoch 11/200, Loss: 0.514488, Accuracy: 0.750000
Epoch 12/200, Loss: 0.456691, Accuracy: 0.843750
Epoch 13/200, Loss: 0.560853, Accuracy: 0.718750
Epoch 14/200, Loss: 0.430779, Accuracy: 0.843750
Epoch 15/200, Loss: 0.561482, Accuracy: 0.781250
Epoch 16/200, Loss: 0.528940, Accuracy: 0.781250
Epoch 17/200, Loss: 0.397544, Accuracy: 0.812500
Epoch 18/200, Loss: 0.459164, Accuracy: 0.750000
Epoch 19/200, Loss: 0.546700, Accuracy: 0.718750
Epoch 20/200, Loss: 0.451045, Accuracy: 0.781250
Epoch 21/200, Loss: 0.497783,